In [14]:
pd.set_option('display.max_colwidth', None)

In [15]:
import pandas as pd

csv_file = "tour_link.csv"
df = pd.read_csv(csv_file)

df

,Country,City,Tour Category,Link
0,영국,런던,시내투어,https://www.myrealtrip.com/offers?t=llp&qct=London&qcr=United%20Kingdom&ext_categories=sinae_tour
1,영국,런던,근교투어,https://www.myrealtrip.com/offers?t=llp&qct=London&qcr=United%20Kingdom&ext_categories=suburb_tour
2,영국,런던,미술관·박물관투어,https://www.myrealtrip.com/offers?t=llp&qct=London&qcr=United%20Kingdom&ext_categories=artmuseum_tour
3,영국,런던,체험·클래스,https://www.myrealtrip.com/offers?t=llp&qct=London&qcr=United%20Kingdom&ext_categories=activity_class
4,영국,런던,투어,https://www.myrealtrip.com/offers?t=llp&qct=London&qcr=United%20Kingdom&ext_categories=tour


In [32]:
import requests
import json


base_url = df['Link'][0]
PER_PAGE = 24

api_url = base_url.replace("https://www.myrealtrip.com/offers", 
                               "https://api3.myrealtrip.com/search/ticket/v2/web") + f"&per={PER_PAGE}" + "&page=30"

response = requests.get(api_url)

json_data = response.json()

if "offers" not in json_data['data'] or not json_data['data']["offers"]:
    print("No data")
else:
    print("Data exists in this page.")

No data


In [ ]:
json_data['data']['offers']

In [33]:
import requests
import pandas as pd
import time

# CSV 파일 로드
csv_file = "tour_link.csv"
df = pd.read_csv(csv_file)

# 결과 저장을 위한 리스트
tours = []

# 한 페이지에 포함될 최대 상품 개수
PER_PAGE = 24

# 각 행을 순회하며 데이터 수집
for _, row in df.iterrows():
    country = row["Country"]
    city = row["City"]
    category = row["Tour Category"]
    base_url = row["Link"]

    # API 요청 링크 생성
    api_url = base_url.replace("https://www.myrealtrip.com/offers", 
                               "https://api3.myrealtrip.com/search/ticket/v2/web") + f"&per={PER_PAGE}"

    page = 1
    while True:
        url = f"{api_url}&page={page}"
        print(f"Fetching: {url}")  # 진행상황 확인

        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch data from {url}")
            break

        data = response.json()

        # 데이터가 없으면 종료
        if "offers" not in data['data'] or not data['data']["offers"]:
            break

        # 필요한 정보 추출
        for offers in data['data']["offers"]:
            tours.append({
                "Country": country,
                "City": city,
                "Category": category,
                "Product ID": offers.get("productId"),
                "Product Type": offers.get("productType"),
                "Title": offers.get("title"),
                "Review Count": offers.get("reviewCount", 0),
                "Review Score": offers.get("reviewScore", 0.0),
                "Original Price": offers.get("originalPrice"),
                "Sale Price": offers.get("salePrice"),
                "Currency": offers.get("currency"),
                "Link": offers.get("linkUrl")
            })

        page += 1  # 다음 페이지로 이동
        time.sleep(0.5)  # 서버 부하 방지를 위해 1초 대기

# 결과를 DataFrame으로 변환 후 CSV 저장
myrealtrip_products = pd.DataFrame(tours)
myrealtrip_products.to_csv("myrealtrip_products.csv", index=False)

print("Data collection complete. Saved to myrealtrip_products.csv.")


Fetching: https://api3.myrealtrip.com/search/ticket/v2/web?t=llp&qct=London&qcr=United%20Kingdom&ext_categories=sinae_tour&per=24&page=1
Fetching: https://api3.myrealtrip.com/search/ticket/v2/web?t=llp&qct=London&qcr=United%20Kingdom&ext_categories=sinae_tour&per=24&page=2
Fetching: https://api3.myrealtrip.com/search/ticket/v2/web?t=llp&qct=London&qcr=United%20Kingdom&ext_categories=sinae_tour&per=24&page=3
Fetching: https://api3.myrealtrip.com/search/ticket/v2/web?t=llp&qct=London&qcr=United%20Kingdom&ext_categories=sinae_tour&per=24&page=4
Fetching: https://api3.myrealtrip.com/search/ticket/v2/web?t=llp&qct=London&qcr=United%20Kingdom&ext_categories=sinae_tour&per=24&page=5
Fetching: https://api3.myrealtrip.com/search/ticket/v2/web?t=llp&qct=London&qcr=United%20Kingdom&ext_categories=sinae_tour&per=24&page=6
Fetching: https://api3.myrealtrip.com/search/ticket/v2/web?t=llp&qct=London&qcr=United%20Kingdom&ext_categories=sinae_tour&per=24&page=7
Fetching: https://api3.myrealtrip.com/sea

In [7]:
data

{'data': {'type': 'llp',
  'cityInfo': {'id': 7, 'keyName': 'London', 'name': '런던'},
  'countryInfo': {'id': 227, 'keyName': 'United Kingdom', 'name': '영국'},
  'offers': [{'gid': 3426919,
    'systemProvider': 'EXP',
    'productNo': '3426919',
    'productId': '3426919',
    'status': 'ONSALE',
    'title': '투어같지 않은 런던 야경투어, 맥주 한잔과 유람선까지 by 헌스트립스',
    'unit': '명',
    'currency': 'KRW',
    'reviewCount': 1889,
    'reviewScore': 4.8,
    'originalPrice': 45000,
    'salePrice': 45000,
    'couponAppliedPrice': None,
    'discountRate': '',
    'isWish': 'UNSELECTED',
    'category': {'code': 'sinae_tour', 'name': '시내투어'},
    'country': {'keyName': 'United Kingdom', 'name': '영국'},
    'city': {'keyName': 'London', 'name': '런던'},
    'imageUrls': {'small': 'https://dry7pvlp22cox.cloudfront.net/mrt-images-prod/2024/07/17/pcs5/aRjOFgO3ba.jpg?width=200&height=150',
     'original': 'https://dry7pvlp22cox.cloudfront.net/mrt-images-prod/2024/07/17/pcs5/aRjOFgO3ba.jpg',
     'large': 'http